In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
import pandas as pd
import numpy as np
import torch
import collections, re
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.util import ngrams
from nltk.tokenize import RegexpTokenizer
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from keras.layers import  CuDNNLSTM 
from keras.layers import concatenate
from keras import Sequential, Model
from sklearn.metrics import classification_report, confusion_matrix,f1_score,accuracy_score

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
clean_data=pd.read_csv("/content/drive/My Drive/NLP project/final_clean_data.csv",encoding='latin-1') #you need to change the path

In [5]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

train_indices, test_indices, y_train, y_test = train_test_split(np.arange(len(clean_data)), clean_data['fraudulent'],
                                                    stratify=clean_data['fraudulent'], 
                                                    test_size=0.3,random_state=0)
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(np.array(train_indices).reshape(-1, 1), clean_data.iloc[train_indices]['fraudulent'])
X_resampled=X_resampled.flatten()

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [0]:
train_indices=X_resampled

In [7]:
clean_data.head()

,job_id,title,company_profile,description,Special_Char,telecommuting,has_company_logo,has_questions,Ambiguous Range,missing_salary,missing_stat,ambiguous_salary,missing,major_benefits,Country_AU,Country_GB,Country_MY,Country_US,State_CA,State_MD,State_NSW,State_NY,State_None,State_TX,employment_type_Contract,employment_type_Full-time,employment_type_None,employment_type_Part-time,required_experience_Associate,required_experience_Entry level,required_experience_Mid-Senior level,required_experience_None,required_education_Bachelor & Assoc,required_education_High School,required_education_Master & Doctorate,required_education_None,required_education_Prof & Cert,industry_accounting,industry_computer networking,industry_consumer services,industry_financial services,industry_hospital health care,industry_information technology services,industry_leisure travel tourism,industry_marketing advertising,industry_none,industry_oil energy,industry_real estate,industry_telecommunications,function_accounting auditing,function_administrative,function_customer service,function_engineering,function_information technology,function_none,function_other,function_sales,fraudulent
0,1,marketing intern,food weve created groundbreaking awardwinning ...,food fastgrowing james beard awardwinning onli...,0,0,1,0,1.0,1,0.0,0.0,1.0,0.0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,2,customer service cloud video production,seconds worlds cloud video production service ...,organised focused vibrant awesomedo passion cu...,0,0,1,0,1.0,1,1.0,1.0,0.0,0.0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
2,3,commissioning machinery assistant cma,valor services provides workforce solutions me...,client located houston actively seeking experi...,0,0,1,0,1.0,1,1.0,1.0,1.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
3,4,account executive washington dc,passion improving quality life geography heart...,company esri environmental systems research in...,0,0,1,0,1.0,1,0.0,0.0,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,5,bill review manager,spotsource solutions llc global human capital ...,job title itemization review managerlocation f...,0,0,1,1,1.0,1,1.0,1.0,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
non_text=df1 = clean_data.drop(['title', 'job_id', 'company_profile','description','fraudulent'], axis=1).to_numpy()

In [0]:
clean_data[['company_profile','description']]=clean_data[['company_profile','description']].replace({np.NAN: "Not provided"})
clean_data['company_profile']=clean_data['company_profile'].astype(str)
clean_data['description']=clean_data['description'].astype(str)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=3000, ngram_range=(1,1))
X1 = vectorizer.fit_transform(clean_data["company_profile"]).toarray()
X1_train=X1[train_indices]
X1_test=X1[test_indices]
Y_train=clean_data['fraudulent'][train_indices]
Y_test=clean_data['fraudulent'][test_indices]

X2 = vectorizer.fit_transform(clean_data["description"]).toarray()
X2_train=X2[train_indices]
X2_test=X2[test_indices]
Y_train=clean_data['fraudulent'][train_indices]
Y_test=clean_data['fraudulent'][test_indices]

In [0]:
Y1 = pd.get_dummies(clean_data['fraudulent']).values
Y1_train=Y1[train_indices]
Y1_test=Y1[test_indices]

In [28]:
max_fatures = 30000
model1 = Sequential()
model1.add(Dense(200,activation='softmax',input_shape = (X1.shape[1],)))
model1.add(Dense(50,activation='softmax'))
model1.add(Dense(2,activation='softmax'))

model2 = Sequential()
model2.add(Dense(200,activation='softmax',input_shape = (X2.shape[1],)))
model2.add(Dense(50,activation='softmax'))
model2.add(Dense(2,activation='softmax'))

model3 = Sequential()
model3.add(Dense(80,input_dim=non_text.shape[1],activation='relu'))
model3.add(Dense(40,activation='relu'))
model3.add(Dense(2,activation='sigmoid'))

model_concat = concatenate([model1.output, model2.output, model3.output], axis=-1)
model_concat = Dense(40, activation='sigmoid')(model_concat)
model_concat = Dense(2, activation='softmax')(model_concat)
model= Model(inputs=[model1.input, model2.input, model3.input], outputs=model_concat)
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())
history=model.fit([X1_train, X2_train,non_text[train_indices]], Y1_train, epochs = 7, batch_size=30,verbose = 2)

y_pred=model.predict([X1_test, X2_test,non_text[test_indices]],verbose = 2, batch_size = 30)
y_pred1=np.argmax(y_pred,axis=1)
Y1_test1=np.argmax(Y1_test,axis=1)
print(f1_score(Y1_test1, y_pred1))

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
dense_79_input (InputLayer)     (None, 3000)         0                                            
__________________________________________________________________________________________________
dense_82_input (InputLayer)     (None, 3000)         0                                            
__________________________________________________________________________________________________
dense_85_input (InputLayer)     (None, 53)           0                                            
__________________________________________________________________________________________________
dense_79 (Dense)                (None, 200)          600200      dense_79_input[0][0]             
____________________________________________________________________________________________

In [29]:
from sklearn.metrics import classification_report, confusion_matrix,f1_score,accuracy_score, precision_score, recall_score
print(f1_score(Y1_test1, y_pred1))
print(confusion_matrix(Y1_test1, y_pred1))
print(accuracy_score(Y1_test1, y_pred1))
print(precision_score(Y1_test1, y_pred1))
print(recall_score(Y1_test1, y_pred1))

0.8166666666666667
[[5080   24]
 [  64  196]]
0.9835943325876212
0.8909090909090909
0.7538461538461538


In [30]:
training_loss = history.history['loss']
epoch_count = range(1, len(training_loss) + 1)
plt.plot(epoch_count, training_loss)
plt.title('Training Loss Plot')
plt.xlabel('Epoch')
plt.ylabel('Training Loss')
plt.show();

In [0]:
train_indices, test_indices, y_train, y_test = train_test_split(np.arange(len(clean_data)), clean_data['fraudulent'],
                                                    stratify=clean_data['fraudulent'], 
                                                    test_size=0.3,random_state=0)
reduced_train_indices, val_indices, y_train, y_test = train_test_split(train_indices, clean_data.loc[train_indices]['fraudulent'],
                                                    stratify=clean_data.loc[train_indices]['fraudulent'], 
                                                    test_size=0.2,random_state=0)

# **Cross Validation**

In [33]:
train_indices, test_indices, y_train, y_test = train_test_split(np.arange(len(clean_data)), clean_data['fraudulent'],
                                                    stratify=clean_data['fraudulent'], 
                                                    test_size=0.3,random_state=0)

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=4,random_state=0)
skf.get_n_splits(train_indices, y_train)
stratified_index=[]
for train_index, test_index in skf.split(train_indices, y_train):
  print("TRAIN:", train_index, "TEST:", test_index)
  stratified_index.append({"train":train_index,"test":test_index})
stratified_index

TRAIN: [ 3125  3126  3127 ... 12513 12514 12515] TEST: [   0    1    2 ... 3173 3176 3207]
TRAIN: [    0     1     2 ... 12513 12514 12515] TEST: [3125 3126 3127 ... 6442 6489 6503]
TRAIN: [    0     1     2 ... 12513 12514 12515] TEST: [6244 6245 6246 ... 9958 9959 9962]
TRAIN: [   0    1    2 ... 9958 9959 9962] TEST: [ 9356  9357  9358 ... 12513 12514 12515]


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


[{'test': array([   0,    1,    2, ..., 3173, 3176, 3207]),
  'train': array([ 3125,  3126,  3127, ..., 12513, 12514, 12515])},
 {'test': array([3125, 3126, 3127, ..., 6442, 6489, 6503]),
  'train': array([    0,     1,     2, ..., 12513, 12514, 12515])},
 {'test': array([6244, 6245, 6246, ..., 9958, 9959, 9962]),
  'train': array([    0,     1,     2, ..., 12513, 12514, 12515])},
 {'test': array([ 9356,  9357,  9358, ..., 12513, 12514, 12515]),
  'train': array([   0,    1,    2, ..., 9958, 9959, 9962])}]

In [34]:
from itertools import product
batch_size = [20, 30, 40]
epochs = [4, 7, 10]
param_grid = list(product(batch_size, epochs))
param_grid

[(20, 4),
 (20, 7),
 (20, 10),
 (30, 4),
 (30, 7),
 (30, 10),
 (40, 4),
 (40, 7),
 (40, 10)]

In [35]:
F_1_cv=[]
embed_dim = 500
lstm_out = 200
from statistics import mean 
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)

for i in param_grid:
  f1=[]
  for j in stratified_index:
    X_resampled, y_resampled = ros.fit_resample(np.array(j["train"]).reshape(-1, 1), clean_data.iloc[j["train"]]['fraudulent'])
    X_resampled=X_resampled.flatten()

    model1 = Sequential()
    model1.add(Dense(200,activation='softmax',input_shape = (X1.shape[1],)))
    model1.add(Dense(50,activation='softmax'))
    model1.add(Dense(2,activation='softmax'))

    model2 = Sequential()
    model2.add(Dense(200,activation='softmax',input_shape = (X2.shape[1],)))
    model2.add(Dense(50,activation='softmax'))
    model2.add(Dense(2,activation='softmax'))

    model3 = Sequential()
    model3.add(Dense(80,input_dim=non_text.shape[1],activation='relu'))
    model3.add(Dense(40,activation='relu'))
    model3.add(Dense(2,activation='sigmoid'))

    model_concat = concatenate([model1.output, model2.output, model3.output], axis=-1)
    model_concat = Dense(40, activation='sigmoid')(model_concat)
    model_concat = Dense(2, activation='softmax')(model_concat)
    model= Model(inputs=[model1.input, model2.input, model3.input], outputs=model_concat)
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    model.fit([X1[X_resampled], X2[X_resampled],non_text[X_resampled]], Y1[X_resampled], epochs = i[1], batch_size=i[0],verbose = 2)

    y_pred=model.predict([X1[j["test"]], X2[j["test"]],non_text[j["test"]]],verbose = 2, batch_size = i[0])
    y_pred1=np.argmax(y_pred,axis=1)
    Y1_test1=np.argmax(Y1[j["test"]],axis=1)
    print(f1_score(Y1_test1, y_pred1))
    f1.append(f1_score(Y1_test1, y_pred1))
  F_1_cv.append([i,sum(f1)/4])

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Epoch 1/4
 - 16s - loss: 0.3700 - accuracy: 0.8606
Epoch 2/4
 - 16s - loss: 0.0465 - accuracy: 0.9875
Epoch 3/4
 - 16s - loss: 0.0111 - accuracy: 0.9984
Epoch 4/4
 - 16s - loss: 0.0048 - accuracy: 0.9993
0.6829268292682926


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Epoch 1/4
 - 18s - loss: 0.3707 - accuracy: 0.8709
Epoch 2/4
 - 19s - loss: 0.1240 - accuracy: 0.9600
Epoch 3/4
 - 17s - loss: 0.0147 - accuracy: 0.9972
Epoch 4/4
 - 17s - loss: 0.0039 - accuracy: 0.9998
0.5494505494505494


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


...

Epoch 1/10
 - 14s - loss: 0.5412 - accuracy: 0.8121
Epoch 2/10
 - 12s - loss: 0.1596 - accuracy: 0.9483
Epoch 3/10
 - 13s - loss: 0.0263 - accuracy: 0.9966
Epoch 4/10
 - 13s - loss: 0.0085 - accuracy: 0.9994
Epoch 5/10
 - 12s - loss: 0.0043 - accuracy: 0.9998
Epoch 6/10
 - 12s - loss: 0.0027 - accuracy: 0.9999
Epoch 7/10
 - 12s - loss: 0.0025 - accuracy: 0.9998
Epoch 8/10
 - 12s - loss: 0.0021 - accuracy: 0.9997
Epoch 9/10
 - 14s - loss: 0.0014 - accuracy: 0.9998
Epoch 10/10
 - 12s - loss: 7.2287e-04 - accuracy: 1.0000
0.5363984674329502


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Epoch 1/10
 - 14s - loss: 0.5033 - accuracy: 0.8074
Epoch 2/10
 - 13s - loss: 0.1572 - accuracy: 0.9451
Epoch 3/10
 - 13s - loss: 0.0272 - accuracy: 0.9965
Epoch 4/10
 - 13s - loss: 0.0100 - accuracy: 0.9987
Epoch 5/10
 - 13s - loss: 0.0065 - accuracy: 0.9989
Epoch 6/10
 - 13s - loss: 0.0043 - accuracy: 0.9991
Epoch 7/10
 - 13s - loss: 0.0023 - accuracy: 0.9994
Epoch 8/10
 - 13s - loss: 0.0024 - accuracy: 0.9995
Epoch 9/10
 - 13s - loss: 0.0014 - accuracy: 0.9994
Epoch 10/10
 - 12s - loss: 8.0291e-04 - accuracy: 1.0000
0.49789029535864976


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Epoch 1/10
 - 14s - loss: 0.5104 - accuracy: 0.8192
Epoch 2/10
 - 13s - loss: 0.1848 - accuracy: 0.9348
Epoch 3/10
 - 12s - loss: 0.0326 - accuracy: 0.9953
Epoch 4/10
 - 12s - loss: 0.0123 - accuracy: 0.9985
Epoch 5/10
 - 13s - loss: 0.0061 - accuracy: 0.9996
Epoch 6/10
 - 12s - loss: 0.0042 - accuracy: 0.9996
Epoch 7/10
 - 12s - loss: 0.0029 - accuracy: 0.9997
Epoch 8/10
 - 12s - loss: 0.0021 - accuracy: 0.9998
Epoch 9/10
 - 13s - loss: 0.0023 - accuracy: 0.9997
Epoch 10/10
 - 12s - loss: 0.0014 - accuracy: 0.9999
0.7194244604316546


In [36]:
F_1_cv

[[(20, 4), 0.6331715601703258],
 [(20, 7), 0.6039199894074025],
 [(20, 10), 0.6161964989994366],
 [(30, 4), 0.6259994457827489],
 [(30, 7), 0.6261473121588976],
 [(30, 10), 0.6515851499701237],
 [(40, 4), 0.6076144292125519],
 [(40, 7), 0.5833560476234876],
 [(40, 10), 0.5988056642963797]]

In [0]:
from keras import backend as K
import tensorflow as tf
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)

In [42]:
F_1_cv=[]
embed_dim = 500
lstm_out = 200
from statistics import mean 
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)

for i in param_grid:
  f1=[]
  for j in stratified_index:
    X_resampled, y_resampled = ros.fit_resample(np.array(j["train"]).reshape(-1, 1), clean_data.iloc[j["train"]]['fraudulent'])
    X_resampled=X_resampled.flatten()

    model1 = Sequential()
    model1.add(Dense(200,activation='softmax',input_shape = (X1.shape[1],)))
    model1.add(Dense(50,activation='softmax'))
    model1.add(Dense(2,activation='softmax'))

    model2 = Sequential()
    model2.add(Dense(200,activation='softmax',input_shape = (X2.shape[1],)))
    model2.add(Dense(50,activation='softmax'))
    model2.add(Dense(2,activation='softmax'))

    model3 = Sequential()
    model3.add(Dense(80,input_dim=non_text.shape[1],activation='relu'))
    model3.add(Dense(40,activation='relu'))
    model3.add(Dense(2,activation='sigmoid'))

    model_concat = concatenate([model1.output, model2.output, model3.output], axis=-1)
    model_concat = Dense(40, activation='sigmoid')(model_concat)
    model_concat = Dense(2, activation='softmax')(model_concat)
    model= Model(inputs=[model1.input, model2.input, model3.input], outputs=model_concat)
    model.compile(loss = f1_loss, optimizer='adam',metrics = ['accuracy'])
    model.fit([X1[X_resampled], X2[X_resampled],non_text[X_resampled]], Y1[X_resampled], epochs = i[1], batch_size=i[0],verbose = 2)

    y_pred=model.predict([X1[j["test"]], X2[j["test"]],non_text[j["test"]]],verbose = 2, batch_size = i[0])
    y_pred1=np.argmax(y_pred,axis=1)
    Y1_test1=np.argmax(Y1[j["test"]],axis=1)
    print(f1_score(Y1_test1, y_pred1))
    f1.append(f1_score(Y1_test1, y_pred1))
  F_1_cv.append([i,sum(f1)/4])

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Epoch 1/4
 - 23s - loss: 0.2316 - accuracy: 0.8480
Epoch 2/4
 - 21s - loss: 0.0287 - accuracy: 0.9802
Epoch 3/4
 - 21s - loss: 0.0058 - accuracy: 0.9964
Epoch 4/4
 - 21s - loss: 0.0037 - accuracy: 0.9972
0.6301369863013699


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Epoch 1/4
 - 23s - loss: 0.2284 - accuracy: 0.8608
Epoch 2/4
 - 22s - loss: 0.0795 - accuracy: 0.9280
Epoch 3/4
 - 22s - loss: 0.0620 - accuracy: 0.9422
Epoch 4/4
 - 22s - loss: 0.0294 - accuracy: 0.9739
0.4912280701754387


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


...

Epoch 1/10
 - 17s - loss: 0.2961 - accuracy: 0.8520
Epoch 2/10
 - 15s - loss: 0.0667 - accuracy: 0.9438
Epoch 3/10
 - 15s - loss: 0.0105 - accuracy: 0.9953
Epoch 4/10
 - 15s - loss: 0.0040 - accuracy: 0.9977
Epoch 5/10
 - 15s - loss: 0.0028 - accuracy: 0.9980
Epoch 6/10
 - 15s - loss: 0.0033 - accuracy: 0.9974
Epoch 7/10
 - 15s - loss: 0.0025 - accuracy: 0.9982
Epoch 8/10
 - 15s - loss: 0.0023 - accuracy: 0.9980
Epoch 9/10
 - 15s - loss: 0.0025 - accuracy: 0.9979
Epoch 10/10
 - 15s - loss: 0.0021 - accuracy: 0.9982
0.45344129554655876


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Epoch 1/10
 - 18s - loss: 0.2972 - accuracy: 0.8262
Epoch 2/10
 - 16s - loss: 0.1101 - accuracy: 0.9044
Epoch 3/10
 - 16s - loss: 0.0594 - accuracy: 0.9467
Epoch 4/10
 - 16s - loss: 0.0101 - accuracy: 0.9930
Epoch 5/10
 - 16s - loss: 0.0082 - accuracy: 0.9931
Epoch 6/10
 - 16s - loss: 0.0072 - accuracy: 0.9936
Epoch 7/10
 - 16s - loss: 0.0066 - accuracy: 0.9940
Epoch 8/10
 - 16s - loss: 0.0046 - accuracy: 0.9958
Epoch 9/10
 - 15s - loss: 0.0030 - accuracy: 0.9972
Epoch 10/10
 - 16s - loss: 0.0025 - accuracy: 0.9981
0.5232067510548524


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Epoch 1/10
 - 18s - loss: 0.3104 - accuracy: 0.8244
Epoch 2/10
 - 15s - loss: 0.0861 - accuracy: 0.9279
Epoch 3/10
 - 15s - loss: 0.0181 - accuracy: 0.9893
Epoch 4/10
 - 15s - loss: 0.0112 - accuracy: 0.9916
Epoch 5/10
 - 15s - loss: 0.0093 - accuracy: 0.9923
Epoch 6/10
 - 15s - loss: 0.0085 - accuracy: 0.9924
Epoch 7/10
 - 15s - loss: 0.0081 - accuracy: 0.9926
Epoch 8/10
 - 15s - loss: 0.0078 - accuracy: 0.9927
Epoch 9/10
 - 15s - loss: 0.0075 - accuracy: 0.9928
Epoch 10/10
 - 15s - loss: 0.0075 - accuracy: 0.9928
0.6250000000000001


In [43]:
F_1_cv

[[(20, 4), 0.554649564514459],
 [(20, 7), 0.5948646797106691],
 [(20, 10), 0.5995398928499013],
 [(30, 4), 0.5701455775068994],
 [(30, 7), 0.5912026456780755],
 [(30, 10), 0.5317729199131034],
 [(40, 4), 0.567531427034122],
 [(40, 7), 0.5778589292159744],
 [(40, 10), 0.5381365625485565]]

In [45]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

train_indices, test_indices, y_train, y_test = train_test_split(np.arange(len(clean_data)), clean_data['fraudulent'],
                                                    stratify=clean_data['fraudulent'], 
                                                    test_size=0.3,random_state=0)
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(np.array(train_indices).reshape(-1, 1), clean_data.iloc[train_indices]['fraudulent'])
train_indices=X_resampled.flatten()

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [49]:
max_fatures = 30000
model1 = Sequential()
model1.add(Dense(200,activation='softmax',input_shape = (X1.shape[1],)))
model1.add(Dense(50,activation='softmax'))
model1.add(Dense(2,activation='softmax'))

model2 = Sequential()
model2.add(Dense(200,activation='softmax',input_shape = (X2.shape[1],)))
model2.add(Dense(50,activation='softmax'))
model2.add(Dense(2,activation='softmax'))

model3 = Sequential()
model3.add(Dense(80,input_dim=non_text.shape[1],activation='relu'))
model3.add(Dense(40,activation='relu'))
model3.add(Dense(2,activation='sigmoid'))

model_concat = concatenate([model1.output, model2.output, model3.output], axis=-1)
model_concat = Dense(40, activation='sigmoid')(model_concat)
model_concat = Dense(2, activation='softmax')(model_concat)
model= Model(inputs=[model1.input, model2.input, model3.input], outputs=model_concat)
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())
history=model.fit([X1_train, X2_train,non_text[train_indices]], Y1_train, epochs = 10, batch_size=30,verbose = 2)

y_pred=model.predict([X1_test, X2_test,non_text[test_indices]],verbose = 2, batch_size = 30)
y_pred1=np.argmax(y_pred,axis=1)
Y1_test1=np.argmax(Y1_test,axis=1)
print(f1_score(Y1_test1, y_pred1))

Model: "model_90"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
dense_970_input (InputLayer)    (None, 3000)         0                                            
__________________________________________________________________________________________________
dense_973_input (InputLayer)    (None, 3000)         0                                            
__________________________________________________________________________________________________
dense_976_input (InputLayer)    (None, 53)           0                                            
__________________________________________________________________________________________________
dense_970 (Dense)               (None, 200)          600200      dense_970_input[0][0]            
___________________________________________________________________________________________

In [50]:
from sklearn.metrics import classification_report, confusion_matrix,f1_score,accuracy_score, precision_score, recall_score
print(f1_score(Y1_test1, y_pred1))
print(confusion_matrix(Y1_test1, y_pred1))
print(accuracy_score(Y1_test1, y_pred1))
print(precision_score(Y1_test1, y_pred1))
print(recall_score(Y1_test1, y_pred1))

0.7870967741935484
[[5082   22]
 [  77  183]]
0.9815436241610739
0.8926829268292683
0.7038461538461539


In [51]:
training_loss = history.history['loss']
epoch_count = range(1, len(training_loss) + 1)
plt.plot(epoch_count, training_loss)
plt.title('Training Loss Plot')
plt.xlabel('Epoch')
plt.ylabel('Training Loss')
plt.show();